In [2]:
!pip install -U -q PyDrive

In [3]:
!pip install -U -q pyarrow

In [4]:
# importações necessarias
import pandas as pd
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import warnings
import shutil
warnings.filterwarnings('ignore')
import plotly.express as px
import pyarrow as pa
import pyarrow.parquet as pq

In [5]:
# Autenticação para acesso aos dados
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [6]:
# Download dos dados para esta máquina virtual
shutil.unpack_archive(f"/content/drive/MyDrive/06120018.zip", "./dataset")


In [7]:
def convert_columns(df_filtered):
    for coluna in df_filtered.columns:
        if df_filtered[coluna].dtype == 'float64':
            df_filtered[coluna] = df_filtered[coluna].astype('float32')

        if df_filtered[coluna].dtype == 'int64':
            df_filtered[coluna] = df_filtered[coluna].astype('int32')

    return df_filtered

In [8]:
filtered_columns = ['recording_time', 'dateDay-1', 'dateMonth-1', 'dateYear-1', 'phaseOfFlight-1',
                 'message0418DAA-1','message0422DAA-1',
                 'bleedFavTmCmd-1a', 'bleedFavTmCmd-1b', 'bleedFavTmFbk-1a',
                 'bleedFavTmFbk-1b', 'bleedHprsovCmdStatus-1a',
                 'bleedHprsovCmdStatus-1b',
                  'bleedHprsovOpPosStatus-1a',
                 'bleedHprsovOpPosStatus-1b',
                  'bleedMonPress-1a',
                 'bleedMonPress-1b',
                 'bleedOnStatus-1a', 'bleedOnStatus-1b',
                 'bleedPrecoolDiffPress-1a', 'bleedPrecoolDiffPress-1b',
                 'bleedPrsovClPosStatus-1a',
                 'bleedPrsovFbk-1a']

In [9]:
dir_path = '/content/dataset/06120018'
print(len([entry for entry in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, entry))]))

2534


In [23]:
import os
import pandas as pd
from datetime import datetime, timedelta

def find_and_store_first_failure_with_range(folder_path):
    for _, _, arquivos in os.walk(folder_path):
        for arquivo in arquivos:
            try:
                path_air_falha = os.path.join(folder_path, arquivo)
                df_read = pd.read_parquet(path_air_falha, engine='auto')

                if len(df_read['message0418DAA-1'].value_counts()) > 1 or len(df_read['message0422DAA-1'].value_counts()) > 1:
                    data = df_read.loc[3, 'dateDay-1'], df_read.loc[3, 'dateMonth-1'], df_read.loc[3, 'dateYear-1']
                    falha_info = {
                        "path": path_air_falha,
                        "mes": int(data[1]),
                        "dia": int(data[0]),
                        "ano": int(data[2])
                    }
                    print(f"Arquivo com falha encontrado: {path_air_falha}")
                    return falha_info

            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {str(e)}")

    print("Nenhum arquivo com falhas encontrado.")
    return None

def get_parquets_in_date_range(folder_path, target_date):
    parquet_files = []

    for _, _, arquivos in os.walk(folder_path):
        for arquivo in arquivos:
            try:
                path_air_falha = os.path.join(folder_path, arquivo)
                df_read = pd.read_parquet(path_air_falha, engine='auto')

                data_day = df_read.loc[3, 'dateDay-1']
                data_month = df_read.loc[3, 'dateMonth-1']
                data_year = df_read.loc[3, 'dateYear-1']

                # Verifica se os valores são numéricos e não são NaN
                if pd.notna(data_day) and pd.notna(data_month) and pd.notna(data_year):
                    try:
                        data_day = int(data_day)
                        data_month = int(data_month)
                        data_year = int(data_year)
                    except ValueError:
                        print(f"Erro ao converter valores de data para inteiros no arquivo {arquivo}")
                        continue

                    file_date = datetime(data_year, data_month, data_day)

                    # Calcula a diferença de dias entre a data do arquivo e a data do erro
                    days_difference = (target_date - file_date).days

                    print(f"Data do arquivo: {file_date.strftime('%d/%m/%Y')}, Diferença de dias: {days_difference}")

                    # Verifica se a diferença está dentro do intervalo de 7 a 14 dias e se a data é anterior à data do erro
                    if 7 <= days_difference <= 14 and file_date < target_date:
                        parquet_files.append(path_air_falha)

            except Exception as e:
                print(f"Erro ao ler o arquivo {arquivo}: {str(e)}")

    return parquet_files

folder_path = '/content/dataset/06120018/'
falha_encontrada = find_and_store_first_failure_with_range(folder_path)

if falha_encontrada:
    print(f"Caminho: {falha_encontrada['path']}, Data: {falha_encontrada['dia']}/{falha_encontrada['mes']}/{falha_encontrada['ano']}")

    # Calcula a data alvo para a pesquisa de arquivos no intervalo de 7 a 14 dias antes da data do erro
    target_date = datetime(falha_encontrada['ano'], falha_encontrada['mes'], falha_encontrada['dia'])

    # Obtém os arquivos Parquet no intervalo de 7 a 14 dias antes da data do erro e anteriores à data do erro
    parquet_files_in_range = get_parquets_in_date_range(folder_path, target_date)

    print("Arquivos no intervalo de 7 a 14 dias antes da data do erro e anteriores à data do erro:")
    for parquet_file in parquet_files_in_range:
        print(parquet_file)


Arquivo com falha encontrado: /content/dataset/06120018/TCRF_ARCHIVE_06120018_20230102035947.parquet
Caminho: /content/dataset/06120018/TCRF_ARCHIVE_06120018_20230102035947.parquet, Data: 2/1/2023
Data do arquivo: 31/05/2022, Diferença de dias: 216
Data do arquivo: 29/06/2023, Diferença de dias: -178
Data do arquivo: 09/06/2023, Diferença de dias: -158
Data do arquivo: 23/06/2023, Diferença de dias: -172
Data do arquivo: 22/05/2023, Diferença de dias: -140
Data do arquivo: 12/03/2023, Diferença de dias: -69
Data do arquivo: 20/05/2023, Diferença de dias: -138
Data do arquivo: 08/07/2022, Diferença de dias: 178
Data do arquivo: 29/05/2023, Diferença de dias: -147
Data do arquivo: 20/04/2023, Diferença de dias: -108
Data do arquivo: 09/10/2022, Diferença de dias: 85
Data do arquivo: 14/07/2023, Diferença de dias: -193
Data do arquivo: 23/01/2023, Diferença de dias: -21
Data do arquivo: 25/10/2022, Diferença de dias: 69
Data do arquivo: 16/05/2023, Diferença de dias: -134
Data do arquivo:

In [ ]:
dfs = []
for parquet_file in parquet_files_in_range:
    df_read = pd.read_parquet(parquet_file, engine='auto')
    dfs.append(df_read)

# Concatena os DataFrames em um único DataFrame
combined_df = pd.concat(dfs, ignore_index=True)

# Caminho para salvar o arquivo Parquet combinado
output_path = '/content/combined_parquet.parquet'

# Salva o DataFrame combinado em um único arquivo Parquet
combined_df.to_parquet(output_path, engine='auto', index=False)

print(f"Arquivo Parquet combinado salvo em: {output_path}")